In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from sorted_alpha import sorted_alpha

# the first 2 minutes of each video 

In [ ]:
import os
import pandas as pd

folder_name = '/Users/andrei-macpro/Documents/Data/openpose/play/tracking/tracking/1206_play_1'

tracks = []
f_names = sorted_alpha(folder_name)
for filename in f_names:
    # load the file in a pandas dataframe
    print(filename)
    filepath = os.path.join(folder_name, filename)
    df = pd.read_csv(filepath)
    df = df[~df.index.duplicated(keep='first')]
    tracks.append(df)

# Function to average x, y values for every 50 rows in the first 3000 frames 
def average_xy(df):
    averaged_data = []
    df = df[df.index < 3000]
    
    # Iterate through the DataFrame in chunks of 50 frames
    for i in range(0, len(df), 50):
        chunk = df.iloc[i:i+50]
        
        
        # Calculate the average x and y values for the chunk
        avg_x = chunk['x'].mean()
        avg_y = chunk['y'].mean()
        averaged_data.append({'frame': chunk.index[0], 'avg_x': avg_x, 'avg_y': avg_y})
    
    return pd.DataFrame(averaged_data)

# Apply the function to each DataFrame in tracks
averaged_tracks = [average_xy(df) for df in tracks]

# Print the averaged tracks
for i, avg_df in enumerate(averaged_tracks):
    print(f"Averaged DataFrame {i+1}:\n", avg_df)

# Print the cleaned tracks
for i, df in enumerate(tracks):
    print(f"Cleaned DataFrame {i+1}:\n", df)

In [ ]:
averaged_tracks[0]

In [ ]:
combined = pd.merge(averaged_tracks[0], averaged_tracks[1], on='frame', suffixes=('_caregiver', '_child'))
print("Aligned Positions Data:")
print(combined)

In [ ]:
# Calculate the differences between consecutive rows for 'avg_x_child' and 'avg_y_child'
diff_x = combined['avg_x_child'].diff().dropna()
diff_y = combined['avg_y_child'].diff().dropna()

# Calculate the Euclidean distance
child_movement = np.sqrt(diff_x**2 + diff_y**2)
print("Euclidean differences between consecutive rows for 'avg_x_child' and 'avg_y_child':")
print(euclidean_distances)

In [ ]:
# Calculate the Euclidean distances between avg x and y positions of caregiver and child
diff_x = combined['avg_x_caregiver'] - combined['avg_x_child']
diff_y = combined['avg_y_caregiver'] - combined['avg_y_child']

# Calculate the Euclidean distance
proximity = np.sqrt(diff_x**2 + diff_y**2)
print("Euclidean distances between avg x and y positions of caregiver and child:")
print(euclidean_distances)

In [ ]:
## now for all the files in the folder
from moviepy.editor import VideoFileClip
video_folder = '/Users/andrei-macpro/Documents/Data/videos/play_videos' 

durations = []
file_names = []

for folder_name in sorted(os.listdir(video_folder)):
    if folder_name == ".DS_Store":
        continue
    file_path = os.path.join(video_folder, folder_name)
    if os.path.isfile(file_path):
        clip = VideoFileClip(file_path)
        print(clip.fps)
        durations.append(clip.duration)



In [ ]:
# find a way to align the recordings in terms of time
durations_meal = pd.Series(durations)
durations_meal.sort_values()

In [ ]:
durations_meal.hist()


In [ ]:
# find a way to align the recordings in terms of time
durations_play = pd.Series(durations)
durations_play.sort_values()

In [ ]:
durations_play.hist()

In [ ]:
# the features are child movement, caregiver movement, and proximity 
# it doesn't make sense to align the recordings of the same participant in terms of time
# so find a way to combine them 
# also resample for same fps 


# Dynamic Time Warping

In [ ]:
# resample all to 25 fps
def resample_df(df, original_fps, target_fps):
    # Convert frame indices to time-based index
    df['time'] = pd.to_timedelta(df.index / original_fps, unit='s')
    df.set_index('time', inplace=True)
    
    # Resample the data to the target fps
    resample_interval = f'{int(1e9 / target_fps)}N'  # Nanoseconds interval
    df_resampled = df.resample(resample_interval).mean().dropna()
    
    # Convert time-based index back to frame indices
    df_resampled.index = (df_resampled.index.total_seconds() * target_fps).astype(int)
    return df_resampled

In [ ]:
tracking_folder = '/Users/andrei-macpro/Documents/Data/openpose/play/tracking/tracking/' 
video_folder = '/Users/andrei-macpro/Documents/Data/videos/play_videos'   # Assuming video files 

features_dict = {}
for folder_name in sorted(os.listdir(tracking_folder)):
    if folder_name == ".DS_Store":
        continue
    file_path = os.path.join(tracking_folder, folder_name)
    tracks = {}
    clip = VideoFileClip(os.path.join(video_folder, folder_name+'.mp4',))
    if clip.fps > 25:
        for file in sorted(os.listdir(file_path)):
            if file == ".DS_Store":
                continue    
            filepath = os.path.join(tracking_folder,folder_name, file)
            df = pd.read_csv(filepath, index_col=0)
            df = df[~df.index.duplicated(keep='first')]
            df.columns = ['x_' + file.split('.')[0], 'y_' + file.split('.')[0]]
            df_resampled = resample_df(df, original_fps=clip.fps, target_fps=25)
            tracks[file.split('.')[0]] = df_resampled
        # Combine the resampled DataFrames
        combined = pd.concat([df for df in tracks.values()], axis=1).dropna()
        # Calculate the differences between consecutive rows for 'x_child' and 'y_child'
        diff_x = combined['x_child'].diff().dropna()
        diff_y = combined['y_child'].diff().dropna()
        child_movement = np.sqrt(diff_x**2 + diff_y**2)
        # Calculate the differences between consecutive rows for 'x_cg' and 'y_cg'
        diff_x = combined['x_cg'].diff().dropna()
        diff_y = combined['y_cg'].diff().dropna()
        caregiver_movement = np.sqrt(diff_x**2 + diff_y**2)
        # Calculate proximiy
        diff_x = combined['x_cg'] - combined['x_child']
        diff_y = combined['y_cg'] - combined['y_child']
        proximity = np.sqrt(diff_x**2 + diff_y**2)
        merged_df = pd.concat([child_movement, caregiver_movement, proximity], axis=1).dropna()
        merged_df.columns = ['child_movement', 'caregiver_movement', 'proximity']   
        features_dict[int(folder_name.split('_')[0])] = merged_df
    else:
        for file in sorted(os.listdir(file_path)):
            if file == ".DS_Store":
                continue    
            filepath = os.path.join(tracking_folder,folder_name, file)
            df = pd.read_csv(filepath, index_col=0)
            df = df[~df.index.duplicated(keep='first')]
            df.columns = ['x_' + file.split('.')[0], 'y_' + file.split('.')[0]]
            tracks[file.split('.')[0]] = df
        # Combine the resampled DataFrames
        combined = pd.concat([df for df in tracks.values()], axis=1).dropna()
        # Calculate the differences between consecutive rows for 'x_child' and 'y_child'
        diff_x = combined['x_child'].diff().dropna()
        diff_y = combined['y_child'].diff().dropna()
        child_movement = np.sqrt(diff_x**2 + diff_y**2)
        # Calculate the differences between consecutive rows for 'x_cg' and 'y_cg'
        diff_x = combined['x_cg'].diff().dropna()
        diff_y = combined['y_cg'].diff().dropna()
        caregiver_movement = np.sqrt(diff_x**2 + diff_y**2)
        # Calculate proximiy
        diff_x = combined['x_cg'] - combined['x_child']
        diff_y = combined['y_cg'] - combined['y_child']
        proximity = np.sqrt(diff_x**2 + diff_y**2)
        merged_df = pd.concat([child_movement, caregiver_movement, proximity], axis=1).dropna()
        merged_df.columns = ['child_movement', 'caregiver_movement', 'proximity']   
        features_dict[int(folder_name.split('_')[0])] = merged_df
         





In [ ]:
features_dict

In [ ]:
import numpy as np
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean



# Identify the shortest sequence
shortest_sequence_key = min(features_dict, key=lambda k: len(features_dict[k]))
shortest_sequence = features_dict[shortest_sequence_key]

# Align sequences using DTW
aligned_features_dict = {}

for key, sequence in features_dict.items():
    print(f"Aligning sequence for {key}...")
    if key == shortest_sequence_key:
        aligned_features_dict[key] = sequence
        continue
    
    # Perform DTW alignment
    distance, path = fastdtw(sequence, shortest_sequence, dist=euclidean)

    # Create an aligned sequence based on the DTW path
    aligned_sequence = pd.DataFrame(np.zeros_like(shortest_sequence.values), columns=shortest_sequence.columns)
    for (i, j) in path:
        aligned_sequence.iloc[j] = sequence.iloc[i]
        # Interpolate to fill in any gaps
    aligned_sequence = aligned_sequence.interpolate(method='linear', limit_direction='both')
    aligned_features_dict[key] = aligned_sequence

# Print the aligned features
for key, aligned_sequence in aligned_features_dict.items():
    print(f"Aligned sequence for {key}:")
    print(aligned_sequence)

In [ ]:
# turn all the sequences into numpy arrays then use PCA




In [ ]:
next(iter(aligned_features_dict.values()))

In [ ]:
list(aligned_features_dict.values())[-1]

In [ ]:
shortest_sequence_key